In [1]:
!pip install numpy
!pip install matplotlib
!pip install scikit-learn
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2021-04-13 01:21:04--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.26
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.26|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-04-13 01:21:04--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1593 (1.6K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.56K  --.-KB/s    in 0s      

2021-04-13 01:21:05 (34.0 MB/s) - written to stdout [1593/1593]

setup Colab 

In [2]:
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import lit
# from pyspark.sql.functions import desc
# from pyspark.ml.evaluation import RegressionEvaluator
# from pyspark.ml.recommendation import ALS
# from pyspark import SparkContext as sc
# from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import udf,col
import os
# tools
import re
import math
import json
import requests
import itertools
import numpy as np
import pandas as pd
import time
from datetime import datetime, timedelta
import string

In [3]:
# from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
# from pyspark.mllib.util import MLUtils
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
#from pyspark.sql.functions import split as splitsp
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# %matplotlib inline
# import numpy as np
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

# from pyspark.mllib.linalg import Vectors
# from pyspark.mllib.linalg.distributed import RowMatrix

In [4]:
'''
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark Naive Bayes") \
        .getOrCreate()
    return spark
spark = init_spark()
'''
import sparknlp
spark = sparknlp.start()
#spark = SparkSession.builder.appName("Spark NLP").master("local[4]").config("spark.driver.memory","16G").config("spark.driver.maxResultSize", "0").config("spark.kryoserializer.buffer.max", "2000M").config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.0.1").getOrCreate()

In [5]:
'''
Give a class label to each post. 0 for >= 6% growth, otherwise 1.
'''
from pyspark.sql.types import IntegerType
data = spark.read.csv("100days-wsbdata.csv", header=True)

function = udf(lambda growth: 0 if float(growth[:-1]) >=6 else 1, IntegerType())
data = data.filter(data.growth != "N/A")
data = data.withColumn('label', function(data.growth))
print(data.count())
data.show()

1304
+------+------+-------+--------------------+----------+-----+------------+--------------------+------------+--------------------+----------+--------------------+----------+-----+
|    id|ticker| growth|               title|     flair|score|upvote_ratio|              author|num_comments|                text|   created|                 url| timestamp|label|
+------+------+-------+--------------------+----------+-----+------------+--------------------+------------+--------------------+----------+--------------------+----------+-----+
|krx451|   GME|811.69%|k gme sharesoptio...|      YOLO|   14|        0.94|  Hank-TheSpank-Hill|          42|     sharesoptions k|2021-01-06|https://www.reddi...|1609909200|    0|
|ks1tzw|   GME|811.69%|           gme🚀🚀🚀|Discussion|   28|        0.94|  PicksburghStillers|          37|all right all of ...|2021-01-06|https://www.reddi...|1609909200|    0|
|ksjhuu|  MGNI| 47.13%|$mgni 🌈🐻 have c...|        DD|    7|           1|     420No_Ragrets69|          

In [6]:
from sparknlp.pretrained import PretrainedPipeline
from sparknlp import Finisher
from pyspark.ml import Pipeline

In [7]:
finisher = Finisher().setInputCols(["token", "lemmas", "pos"])
explain_pipeline_model = PretrainedPipeline("explain_document_ml").model

pipeline = Pipeline() \
    .setStages([
        explain_pipeline_model,
        finisher
        ])

model = pipeline.fit(data.select('id', 'label', 'text'))
annotations_finished_df = model.transform(data.select('id','label', 'text'))

explain_document_ml download started this may take some time.
Approx size to download 9.1 MB
[OK!]


In [8]:
from pyspark.sql.types import BooleanType
text_lemmas = annotations_finished_df.select('id', 'label', 'finished_lemmas')
# remove posts for which the text is too short
function_length = udf(lambda r: len(r) > 10, BooleanType())
text_lemmas = text_lemmas.withColumn('word_count', function_length('finished_lemmas')).filter(col('word_count')).drop('word_count')
print("Total posts:", text_lemmas.count())
text_lemmas.show()

Total posts: 1144
+------+-----+--------------------+
|    id|label|     finished_lemmas|
+------+-----+--------------------+
|ks1tzw|    0|[all, right, all,...|
|ksjhuu|    0|[bldr, $, pt, pos...|
|kt79b2|    0|[we, might, be, h...|
|kt9enf|    0|[bear, feast, on,...|
|ktbu75|    0|[so, today, i, go...|
|ktfori|    0|[what, a, week, i...|
|kup2e2|    1|[start, to, feel,...|
|kurzyz|    0|[i, honestly, don...|
|kusng4|    0|[soi, recently, o...|
|kuku2g|    1|[nio, be, disrupt...|
|kutslm|    1|[listen, up, you,...|
|kv7k8k|    0|[have, anyone, no...|
|kvarzs|    0|[now, that, the, ...|
|kvcard|    0|[we, all, know, t...|
|kvcmhk|    1|[it, do, not, dil...|
|kva2kt|    0|[we, know, you, l...|
|kvacad|    1|[so, i, hear, you...|
|kvcimd|    1|[look, at, hour, ...|
|kvtk4b|    1|[giga, berain, co...|
|kw2vwm|    1|[see, the, end, o...|
+------+-----+--------------------+
only showing top 20 rows



In [9]:
'''
Make lemmas writable to CSV
'''
output = text_lemmas.rdd.map(lambda row: Row(id=row['id'], label=row['label'], text='|'.join(row['finished_lemmas'])))
output_df = output.toDF()
output_df.show()

+------+-----+--------------------+
|    id|label|                text|
+------+-----+--------------------+
|ks1tzw|    0|all|right|all|of|...|
|ksjhuu|    0|bldr|$|pt|positio...|
|kt79b2|    0|we|might|be|hear|...|
|kt9enf|    0|bear|feast|on|low...|
|ktbu75|    0|so|today|i|go|to|...|
|ktfori|    0|what|a|week|in|st...|
|kup2e2|    1|start|to|feel|rea...|
|kurzyz|    0|i|honestly|dont|e...|
|kusng4|    0|soi|recently|open...|
|kuku2g|    1|nio|be|disrupt|te...|
|kutslm|    1|listen|up|you|deg...|
|kv7k8k|    0|have|anyone|notic...|
|kvarzs|    0|now|that|the|boy|...|
|kvcard|    0|we|all|know|that|...|
|kvcmhk|    1|it|do|not|dilute|...|
|kva2kt|    0|we|know|you|love|...|
|kvacad|    1|so|i|hear|you|deg...|
|kvcimd|    1|look|at|hour|char...|
|kvtk4b|    1|giga|berlins|come...|
|kw2vwm|    1|see|the|end|of|th...|
+------+-----+--------------------+
only showing top 20 rows



In [10]:
'''
Write lemmas to CSV
'''
output_df.coalesce(1).write.csv('lemma-100daysdata.csv', header=True)